In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QHBoxLayout, QPushButton, QTextEdit, QLabel, QFileDialog, QLineEdit
from PyQt5.QtGui import QPixmap
from PIL import Image
import numpy as np
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import base64

class AESLSBSteganographyGUI(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        self.setWindowTitle('AES Encryption and LSB Steganography')
        self.setGeometry(100, 100, 800, 600)

        layout = QVBoxLayout()

        # Image selection
        self.image_label = QLabel()
        self.image_label.setFixedSize(300, 300)
        self.select_image_button = QPushButton('Select Image')
        self.select_image_button.clicked.connect(self.select_image)
        layout.addWidget(self.image_label)
        layout.addWidget(self.select_image_button)

        # Message input
        self.message_input = QTextEdit()
        self.message_input.setPlaceholderText("Enter message to hide")
        layout.addWidget(QLabel("Message:"))
        layout.addWidget(self.message_input)

        # Key input
        key_layout = QHBoxLayout()
        self.key_input = QLineEdit()
        self.key_input.setPlaceholderText("Enter 16-character key for AES")
        key_layout.addWidget(QLabel("AES Key:"))
        key_layout.addWidget(self.key_input)
        layout.addLayout(key_layout)

        # Buttons
        button_layout = QHBoxLayout()
        self.hide_button = QPushButton('Encrypt and Hide Message')
        self.extract_button = QPushButton('Extract and Decrypt Message')
        self.hide_button.clicked.connect(self.encrypt_and_hide)
        self.extract_button.clicked.connect(self.extract_and_decrypt)
        button_layout.addWidget(self.hide_button)
        button_layout.addWidget(self.extract_button)
        layout.addLayout(button_layout)

        # Output
        self.output_text = QTextEdit()
        self.output_text.setReadOnly(True)
        layout.addWidget(QLabel("Output:"))
        layout.addWidget(self.output_text)

        self.setLayout(layout)

    def select_image(self):
        file_name, _ = QFileDialog.getOpenFileName(self, "Select Image", "", "Image Files (*.png *.jpg *.bmp)")
        if file_name:
            pixmap = QPixmap(file_name)
            self.image_label.setPixmap(pixmap.scaled(self.image_label.size(), aspectRatioMode=True))
            self.image_path = file_name

    def encrypt_aes(self, message, key):
        cipher = AES.new(key.encode('utf-8'), AES.MODE_ECB)
        padded_message = pad(message.encode('utf-8'), AES.block_size)
        encrypted_message = cipher.encrypt(padded_message)
        return base64.b64encode(encrypted_message).decode('utf-8')

    def decrypt_aes(self, encrypted_message, key):
        cipher = AES.new(key.encode('utf-8'), AES.MODE_ECB)
        decrypted_message = cipher.decrypt(base64.b64decode(encrypted_message))
        return unpad(decrypted_message, AES.block_size).decode('utf-8')

    def hide_message_in_image(self, image, message):
        img_array = np.array(list(image.getdata()))

        if image.mode == 'RGB':
            n = 3
        elif image.mode == 'RGBA':
            n = 4

        total_pixels = img_array.size // n

        message += "###"
        b_message = ''.join([format(ord(i), "08b") for i in message])
        req_pixels = len(b_message)

        if req_pixels > total_pixels:
            raise ValueError("Message too large for the image")

        index = 0
        for p in range(total_pixels):
            for q in range(0, 3):
                if index < req_pixels:
                    img_array[p][q] = int(bin(img_array[p][q])[2:9] + b_message[index], 2)
                    index += 1

        img_array = img_array.reshape(image.size[1], image.size[0], n)
        return Image.fromarray(img_array.astype('uint8'), image.mode)

    def extract_message_from_image(self, image):
        img_array = np.array(list(image.getdata()))

        if image.mode == 'RGB':
            n = 3
        elif image.mode == 'RGBA':
            n = 4

        total_pixels = img_array.size // n

        hidden_bits = ""
        for p in range(total_pixels):
            for q in range(0, 3):
                hidden_bits += (bin(img_array[p][q])[2:][-1])

        hidden_bits = [hidden_bits[i:i+8] for i in range(0, len(hidden_bits), 8)]
        message = ""
        for i in range(len(hidden_bits)):
            if message[-3:] == "###":
                break
            else:
                message += chr(int(hidden_bits[i], 2))

        if "###" in message:
            return message[:-3]
        else:
            raise ValueError("No hidden message found")

    def encrypt_and_hide(self):
        if not hasattr(self, 'image_path'):
            self.output_text.setText("Please select an image first.")
            return

        message = self.message_input.toPlainText()
        key = self.key_input.text()

        if not message or len(key) != 16:
            self.output_text.setText("Please enter a message and a 16-character key.")
            return

        try:
            encrypted_message = self.encrypt_aes(message, key)
            img = Image.open(self.image_path)
            stego_img = self.hide_message_in_image(img, encrypted_message)
            stego_img.save("stego_image_encrypted.png")
            self.output_text.setText("Message encrypted and hidden successfully. Saved as 'stego_image_encrypted.png'")
        except Exception as e:
            self.output_text.setText(f"An error occurred: {str(e)}")

    def extract_and_decrypt(self):
        if not hasattr(self, 'image_path'):
            self.output_text.setText("Please select an image first.")
            return

        key = self.key_input.text()

        if len(key) != 16:
            self.output_text.setText("Please enter a 16-character key.")
            return

        try:
            img = Image.open(self.image_path)
            extracted_message = self.extract_message_from_image(img)
            decrypted_message = self.decrypt_aes(extracted_message, key)
            self.output_text.setText(f"Extracted and decrypted message: {decrypted_message}")
        except Exception as e:
            self.output_text.setText(f"An error occurred: {str(e)}")

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = AESLSBSteganographyGUI()
    ex.show()
    sys.exit(app.exec_())